<h1>Исследование надёжности заёмщиков</h1>

<h2>Описание и цели проекта</h2>

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

<h2>Описание данных</h2>

Файл с данными _/datasets/data.csv_ содержит необходимую информацию:

* _children_ — количество детей в семье
* _days_employed_ — общий трудовой стаж в днях
* _dob_years_ — возраст клиента в годах
* _education_ — уровень образования клиента
* _education_id_ — идентификатор уровня образования
* _family_status_ — семейное положение
* _family_status_id_ — идентификатор семейного положения
* _gender_ — пол клиента
* _income_type_ — тип занятости
* _debt_ — имел ли задолженность по возврату кредитов
* _total_income_ — ежемесячный доход
* _purpose_ — цель получения кредита


<h2>Оглавление</h2>

1. [Открытие данных](#step_1)
2. [Предобработка данных](#step_2)
    * [Обработка пропусков](#step_2_null)
    * [Замена типа данных](#step_2_astype)
    * [Обработка дубликатов](#step_2_duplicates)
    * [Лемматизация](#step_2_lemmas)
    * [Категоризация данных](#step_2_categorize)
3. [Вопросы](#step_3)
4. [Общий вывод](#step_4)

### Шаг 1. Откройте файл с данными и изучите общую информацию. 
<a id="step_1"></a>

In [1]:
import pandas as pd

In [2]:
clients = pd.read_csv('/datasets/data.csv')

In [3]:
# посмотрим сначала на первые 5 строк
clients.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [4]:
# узнаем размер таблицы
clients.shape

(21525, 12)

In [5]:
# общая информация по таблице
clients.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


По общей информации можно сказать следующее:
- Есть пропуски в полях days_employed и total_income
- Не подходящий тип данных для полей days_employed и total_income (кол-во дней будем хранить в int64, в ежемесячном доходе наврятли будут интересовать копейки, тоже будем хранить в int64)

In [6]:
# основные статистичекие данные
clients.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


__Основные проблемы уже видно:__
- Кол-во детей в семье содержит отрицательные числа
- Стаж в днях содержит отрицательные числа
- Стаж в днях содержит слибок большие числа для стажа
- Возраст клиента заполнен у всех, но есть некорректные значения (min=0)


In [7]:
# посмотрим категориальные переменные
clients['education'].value_counts()

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64

__Образование__ содержит значения в разных регистрах

In [8]:
clients['family_status'].value_counts()

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

С __семейным положением__ все ок

In [9]:
clients['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

__Пол__ содержит всего одно неопредленное значение XNA

In [10]:
clients['income_type'].value_counts()

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
предприниматель        2
безработный            2
студент                1
в декрете              1
Name: income_type, dtype: int64

__Тип занятости__ корректно заполнен

In [11]:
clients['purpose'].value_counts()

свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
покупка жилья для сдачи                   653
операции с жильем                         653
операции с коммерческой недвижимостью     651
покупка жилья                             647
жилье                                     647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
покупка своего жилья                      620
строительство недвижимости                620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

__Цель кредита__ содержит дубли (разная формулировка)

### Вывод

Файл содержит __12 столбцов__ и __21525 строк__.

Основные проблемы которые небходимо будет решить:
- Есть пропуски в полях __days_employed__ и __total_income__, а так же нулевые значения возраста и одно не верное значение пола
- Не подходящий тип данных для полей __days_employed__ и __total_income__ (кол-во дней будем хранить в int64, в ежемесячном доходе наврятли будут интересовать копейки, тоже будем хранить в int64)
- __Кол-во детей__ в семье содержит отрицательные числа
- __Стаж в днях__ содержит отрицательные числа
- __Стаж в днях__ содержит слибок большие числа для стажа
- __Возраст клиента__ заполнен у всех, но есть некорректные значения (min=0)
- __Образование__ содержит значения в разных регистрах
- __Пол__ содержит всего одно неопредленное значение XNA
- __Цель__ кредита содержит дубли (разная формулировка)

### Шаг 2. Предобработка данных
<a id="step_2"></a>

### Обработка пропусков
<a id="step_2_null"></a>

In [12]:
# первая проблема со стажем, посмотрим сколько записей без пропусков и с пропусками
days_employed_true = clients['days_employed'].notnull().sum()
days_employed_false = clients['days_employed'].isnull().sum()
total = clients.shape[0]
print(f'Всего: {total}')
print(f'Заполнено: {days_employed_true} ({days_employed_true / total :.2%})')
print(f'Пропусков: {days_employed_false} ({days_employed_false / total :.2%})')

Всего: 21525
Заполнено: 19351 (89.90%)
Пропусков: 2174 (10.10%)


In [13]:
# вторая задача с пропусками в ежемесячном доходе
total_income_true = clients['total_income'].notnull().sum()
total_income_false = clients['total_income'].isnull().sum()
total = clients.shape[0]
print(f'Всего: {total}')
print(f'Заполнено: {total_income_true} ({total_income_true / total :.2%})')
print(f'Пропусков: {total_income_false} ({total_income_false / total :.2%})')

Всего: 21525
Заполнено: 19351 (89.90%)
Пропусков: 2174 (10.10%)


In [14]:
# проверим не связаны ли эти просуки друг с другом
clients.loc[clients['days_employed'].isnull() & clients['total_income'].isnull(), ['days_employed', 'total_income']].isnull().count()

days_employed    2174
total_income     2174
dtype: int64

__Видим что пропуск идет одновремено и стажа и ежемесячного дохода__

In [15]:
# будем считать что стаж зависит от возраста
def days_employed_median(dob_years):
    return clients[(clients['days_employed'].notnull()) & (clients['dob_years'] == dob_years)].median()

# и применим эту функцию
clients.loc[clients['days_employed'].isnull(), 'days_employed'] = clients.loc[clients['days_employed'].isnull(), 'dob_years'].apply(days_employed_median)

In [16]:
# проверим что все записи о стаже стали заполнены (пропусков нет)
days_employed_true = clients['days_employed'].notnull().sum()
days_employed_false = clients['days_employed'].isnull().sum()
total = clients.shape[0]
print(f'Всего: {total}')
print(f'Заполнено: {days_employed_true} ({days_employed_true / total :.2%})')
print(f'Пропусков: {days_employed_false} ({days_employed_false / total :.2%})')

Всего: 21525
Заполнено: 21525 (100.00%)
Пропусков: 0 (0.00%)


In [17]:
# будем считать что доход в большей степени зависит от типа занятости
def total_income_median(income_type):
    return clients[(clients['days_employed'].notnull()) & (clients['income_type'] == income_type)].median()

# и применим эту функцию
clients.loc[clients['total_income'].isnull(), 'total_income'] = clients.loc[clients['total_income'].isnull(), 'income_type'].apply(total_income_median)

In [18]:
# проверим что все записи о ежемесячном доходе стали заполнены (пропусков нет)
total_income_true = clients['total_income'].notnull().sum()
total_income_false = clients['total_income'].isnull().sum()
total = clients.shape[0]
print(f'Всего: {total}')
print(f'Заполнено: {total_income_true} ({total_income_true / total :.2%})')
print(f'Пропусков: {total_income_false} ({total_income_false / total :.2%})')

Всего: 21525
Заполнено: 21525 (100.00%)
Пропусков: 0 (0.00%)


### Вывод

Два столбца с пропусками:

* столбец со стажем заполнили медианой зависимой от возраста, 
* столбец с доходом заполнили медианой сгруппировав по типу занятости.

### Замена типа данных
<a id="step_2_astype"></a>

In [19]:
# посмотрим какие типы данных не оптимальны для нас
clients.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [20]:
# два столбца days_employed и total_income с вещественным типом, мы сменим тип данных на целочисленный
clients[['days_employed', 'total_income']] = clients[['days_employed', 'total_income']].astype({'days_employed': 'int64', 'total_income': 'int64'})

In [21]:
# проверим что все ок
clients.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


### Вывод

Мы заменили типы данных двух столбцов days_employed и total_income методом astype и передали ему словарь с наименованиями столбцов и целевыми типами к которым надо преобразовать значения.

### Обработка дубликатов
<a id="step_2_duplicates"></a>

In [22]:
# посмотрим первые 5 записей
clients.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,-5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


In [23]:
# посмотрим на уникальные значения в столбце образование
clients['education'].value_counts()

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64

In [24]:
# в столбце образование преобразуем все значения в нижний регистр
clients['education'] = clients['education'].str.lower()

In [25]:
# посмотрим еще раз на полученный результат
clients['education'].value_counts()

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

In [26]:
# избавимся от отрицательных значений в столбцах children и days_employed
clients[['children', 'days_employed']] = clients[['children', 'days_employed']].abs()

In [27]:
# проверим что отрицательных значений больше нет
clients.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,21525.000000,21525.000000,21525.000000,21525.000000,21525.000000,2.152500e+04
mean,0.543275,66873.635447,43.293380,0.817236,0.972544,0.080883,1.652248e+05
std,1.379876,138660.193140,12.574584,0.548138,1.420324,0.272661,9.804367e+04
min,0.000000,24.000000,0.000000,0.000000,0.000000,0.000000,2.066700e+04
25%,0.000000,974.000000,33.000000,1.000000,0.000000,0.000000,1.077980e+05
50%,0.000000,2013.000000,42.000000,1.000000,0.000000,0.000000,1.425940e+05
75%,1.000000,5333.000000,53.000000,1.000000,1.000000,0.000000,1.955490e+05
max,20.000000,401755.000000,75.000000,4.000000,4.000000,1.000000,2.265604e+06


In [28]:
# посчитаем количество дублей
duplicated = clients.duplicated().sum()
total = clients.shape[0]
print(f'Всего: {total}')
print(f'Дублей: {duplicated} ({duplicated / total :.2%})')

Всего: 21525
Дублей: 71 (0.33%)


In [29]:
clients = clients.drop_duplicates().reset_index(drop=True)

In [30]:
# посчитаем количество дублей
duplicated = clients.duplicated().sum()
total = clients.shape[0]
print(f'Всего: {total}')
print(f'Дублей: {duplicated} ({duplicated / total :.2%})')

Всего: 21454
Дублей: 0 (0.00%)


### Вывод

Для начала необходимо избавиться дублей с разными регистрациями в столбцах с категориальными значениями. Они могут появляться в результате ошибочных действий пользователя. Решаем эту проблему приведением всех значений этого столбца в один регистр. 
Дальше необходимо избавиться от отрицательных значений, т.к. опять же пользователь мог вбить значение через дефис "-".
Дальше можно удалить дубли методом drop_duplicates и перестроить индекс reset_index.

### Лемматизация
<a id="step_2_lemmas"></a>

In [31]:
# импортиуем необходимый модуль
from pymystem3 import Mystem
m = Mystem()

In [32]:
# посмотрим на значения в столбце с целями кредита
clients['purpose'].value_counts()

свадьба                                   791
на проведение свадьбы                     768
сыграть свадьбу                           765
операции с недвижимостью                  675
покупка коммерческой недвижимости         661
операции с жильем                         652
покупка жилья для сдачи                   651
операции с коммерческой недвижимостью     650
покупка жилья                             646
жилье                                     646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          624
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

In [33]:
# опишем функцию для лемматизации и применим её методом apply
def lemmatize(text):
    return ''.join(m.lemmatize(text)[:-1])

clients['purpose_lemmatized'] = clients['purpose'].apply(lemmatize)

In [34]:
# посмотрим что получилось
clients['purpose_lemmatized'].value_counts()

автомобиль                                972
свадьба                                   791
на проведение свадьба                     768
сыграть свадьба                           765
операция с недвижимость                   675
покупка коммерческий недвижимость         661
операция с жилье                          652
покупка жилье для сдача                   651
операция с коммерческий недвижимость      650
жилье                                     646
покупка жилье                             646
покупка жилье для семья                   638
строительство собственный недвижимость    635
недвижимость                              633
операция со свой недвижимость             627
строительство жилой недвижимость          624
покупка недвижимость                      621
покупка свой жилье                        620
строительство недвижимость                619
ремонт жилье                              607
покупка жилой недвижимость                606
на покупка свой автомобиль        

### Вывод

В столбце цель кредита текст содержит слова в разной форме. Необходимо их привести к одной форме с помощью лемматизации из модуля pymystem3. Мы делаем функцию к оторая принимает на вход текстовую переменную и лемматизирует этот текст и соединяет с 1 строку. Далее мы эту функцию применяет к столбцу purpose методом apply.

### Категоризация данных
<a id="step_2_categorize"></a>

#### Категоризация целей кредита

In [35]:
# пока разберемся с целями кредита автомобиль, свадьба, образование. Жилье и недвижимость отдельно.
purposes = ['автомобиль', 'свадьба', 'образование', 'жилье', 'недвижимость']

def purpose_category(text):
    for purpose in purposes:
        if purpose in text:
            return purpose

clients['purpose_category'] = clients['purpose_lemmatized'].apply(purpose_category)

In [36]:
# проверим что получилось
clients['purpose_category'].value_counts()

недвижимость    6351
жилье           4460
автомобиль      4306
образование     4013
свадьба         2324
Name: purpose_category, dtype: int64

#### Категоризация клиентов по уровню дохода

In [37]:
# разобьем всех клиентов на 5 равных по кол-ву клиентов групп по уровню дохода
clients['total_income_group'] = pd.qcut(clients['total_income'], 5)

In [38]:
# проверим что получилось (по кол-ву клиентов)
clients['total_income_group'].value_counts()

(214618.2, 2265604.0]    4291
(161335.0, 214618.2]     4291
(98537.6, 132134.4]      4291
(20666.999, 98537.6]     4291
(132134.4, 161335.0]     4290
Name: total_income_group, dtype: int64

#### Категоризация клиентов по наличию детей

In [39]:
# разобьем на две категории: 0 - нет детей, 1 - есть дети
def have_children(children):
    if children > 0:
        return 1
    else:
        return 0
    
clients['have_children'] = clients['children'].apply(have_children)

#### Семейное положение (словарь)

In [40]:
# в данных есть словарь с семейным положением
family_status_dict = clients[['family_status_id', 'family_status']]
family_status_dict = family_status_dict.drop_duplicates().reset_index(drop=True)
family_status_dict

,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,Не женат / не замужем


#### Образование (словарь)

In [41]:
# в данных так же есть словарь с образованием
education_dict = clients[['education_id', 'education']]
education_dict = education_dict.drop_duplicates().reset_index(drop=True)
education_dict

,education_id,education
0,0,высшее
1,1,среднее
2,2,неоконченное высшее
3,3,начальное
4,4,ученая степень


#### Тип занятости

In [42]:
# и я бы выделил словарь с типом занятости
income_type_dict = clients[['income_type']]
income_type_dict = income_type_dict.drop_duplicates().reset_index(drop=True)
income_type_dict['income_type_id'] = income_type_dict.reset_index().index
income_type_dict

,income_type,income_type_id
0,сотрудник,0
1,пенсионер,1
2,компаньон,2
3,госслужащий,3
4,безработный,4
5,предприниматель,5
6,студент,6
7,в декрете,7


### Вывод

Для категоризации данных в столбце с целью кредита мы сделали словать со следующими значениями:
 - автомобиль
 - свадьба
 - образование
 - жилье
 - недвижимость

И сделали функцию которая проверяет вхождение значений из данного словаря с значениями столбцов лематтизированных целей кредитов и подставляет в новый столбец значение из словаря.

Так же мы категоризировали клиентов по уровню дохода на 5 групп с равным количеством клиентов и категоризировали по наличию детей.

В исходных данных я выделил 3 словаря:

* Семейное положение
* Образование
* Тип занятости

### Шаг 3. Ответьте на вопросы
<a id="step_3"></a>

#### Количество должников по наличию детей
<a id="step_3_children"></a>

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [43]:
# сгруппируем датафрейм по наличию детей, посчитаем кол-во заявок и кол-во заявок с клиентами должниками 
children_groupped = clients.groupby('have_children')['debt'].agg(['count', 'sum']).rename(columns={'count':'count', 'sum':'debt_count'})

# посчитаем какую часть от заявок занимают должники
children_groupped['debt_part'] = children_groupped['debt_count'] / children_groupped['count'] * 100

# посчитаем какую часть от всех заявок занимают должники
children_groupped['debt_part_total'] = children_groupped['debt_count'] / clients.shape[0] * 100

children_groupped

,count,debt_count,debt_part,debt_part_total
have_children,,,,
0,14091,1063,7.543822,4.954787
1,7363,678,9.208203,3.160250


### Вывод

По нашим данным зависимость получается следующая:

Наличие детей влияет чуть больше на возврат кредита в срок, около 9.2% клиентов с детьми имеют задолженности и примерно 7.5% клиентов без детей имеют задолженность, что немного меньше клиентов с детьми.

#### Количество должников в зависимости от семейного положения
<a id="step_3_family"></a>

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [44]:
family_groupped = clients.groupby('family_status')['debt'].agg(['count', 'sum']).rename(columns={'count':'count', 'sum':'debt_count'})
family_groupped['debt_part'] = family_groupped['debt_count'] / family_groupped['count'] * 100
family_groupped['debt_part_total'] = family_groupped['debt_count'] / clients.shape[0] * 100

family_groupped

,count,debt_count,debt_part,debt_part_total
family_status,,,,
Не женат / не замужем,2810,274,9.750890,1.277151
в разводе,1195,85,7.112971,0.396197
вдовец / вдова,959,63,6.569343,0.293652
гражданский брак,4151,388,9.347145,1.808521
женат / замужем,12339,931,7.545182,4.339517


### Вывод

Клиентов не женатых/не замужних а так же клиентов в гражданском браке отличает высокий процент (более 9%) имеющихся среди них должников. 

Вдовцы/вдовы и клиенты в разводе оказываются самыми платежеспособными, самый низкий процент доолжников среди них (6-7%).

Женатые/замужние клиенты оказались в середине с показателем 7,5%, но таких клиентов большая часть из общего количества и поэтому примерно 4% должников из вообще всех именно женатые/замужние должники.

#### Количество должников по уровню дохода
<a id="step_3_income"></a>

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [45]:
total_income_groupped = clients.groupby('total_income_group')['debt'].agg(['count', 'sum']).rename(columns={'count':'count', 'sum':'debt_count'})
total_income_groupped['debt_part'] = total_income_groupped['debt_count'] / total_income_groupped['count'] * 100
total_income_groupped['debt_part_total'] = total_income_groupped['debt_count'] / clients.shape[0] * 100

total_income_groupped

,count,debt_count,debt_part,debt_part_total
total_income_group,,,,
"(20666.999, 98537.6]",4291,344,8.016779,1.603431
"(98537.6, 132134.4]",4291,361,8.412957,1.682670
"(132134.4, 161335.0]",4290,375,8.741259,1.747926
"(161335.0, 214618.2]",4291,361,8.412957,1.682670
"(214618.2, 2265604.0]",4291,300,6.991377,1.398341


### Вывод

Уровень дохода не влияет на возврат кредита. Во всех группа процент должников одинаковый, примерно 8-9%

#### Количество должников по целям кредита
<a id="step_3_purpose"></a>

- Как разные цели кредита влияют на его возврат в срок?

In [46]:
purpose_groupped = clients.groupby('purpose_category')['debt'].agg(['count', 'sum']).rename(columns={'count':'count', 'sum':'debt_count'})
purpose_groupped['debt_part'] = purpose_groupped['debt_count'] / purpose_groupped['count'] * 100
purpose_groupped['debt_part_total'] = purpose_groupped['debt_count'] / clients.shape[0] * 100

purpose_groupped

,count,debt_count,debt_part,debt_part_total
purpose_category,,,,
автомобиль,4306,403,9.359034,1.878438
жилье,4460,308,6.905830,1.435630
недвижимость,6351,474,7.463392,2.209378
образование,4013,370,9.220035,1.724620
свадьба,2324,186,8.003442,0.866971


In [47]:
# попробуем цели кредита развернуть в pivot и посчитать кол-во должников по типу занятости в целях док-ва понимания pivot
clients.pivot_table(index=['income_type'], columns='purpose_category', values='debt', aggfunc='sum')

purpose_category,автомобиль,жилье,недвижимость,образование,свадьба
income_type,,,,,
безработный,NaN,1.0,NaN,NaN,NaN
в декрете,1.0,NaN,NaN,NaN,NaN
госслужащий,22.0,15.0,21.0,21.0,7.0
компаньон,85.0,64.0,103.0,72.0,52.0
пенсионер,51.0,40.0,52.0,48.0,25.0
предприниматель,NaN,NaN,0.0,NaN,0.0
сотрудник,244.0,188.0,298.0,229.0,102.0
студент,NaN,NaN,0.0,NaN,NaN


### Вывод

Разные цели кредита по разному влияют на возврат кредита, но разница составляет примерно 3%. 

Худший показатель имеет кредит на автомобиль - 9.4% клиентов с задолженностью.

Лучший показатель у кредита на жилье - 6.9% клиентов с задолженностью.

### Шаг 4. Общий вывод
<a id="step_4"></a>

Для анализа входных данных сначала необходимо было предообработать эти данные.

Например, пришлось заполнить пропуски по стажу медианой в зависимости от возраста, пропуски ежемесячного дохода заполнили просто медианой. Пропусков было около 10% от общего количества.

Так же были дубликаты категориальных значений, мы привели их к одинаковому регистру и удалили все дубли, из было 0.33%.

Необходимо было категоризировать данные о цели кредита для анализа в дальнейшем. Мы лемматизировали все имеющиеся данные и категоризировали по наличию значений из нашего словаря.

Полученные данные уже можно было анализировать на наличие зависимостей.

Можно сказать, что на факт выплаты кредита в срок влияют некоторые зависимости, например, наличие детей влияет на возврат кредита немного больше, около 9.2% клиентов с детьми имеют задолженности и, примерно, 7.5% клиентов без детей имеют задолженность.

Так же клиенты не женатые/не замужние или в гражданском браке чаще становятся должниками, более 9% клиентов. Вдовцы/вдовы оказываются самыми платежеспособными, среди них всего 6-7% клиентов имеют задолженности, но стоит отметить, что таких клиентов значительно меньше чем клиентов с другим семейным положением.

Проверили еще зависимость цели кредита на его возврат в срок. Автомобиль и образование самые рискованные кредиты, так как больше 9% клиентов имеют задолженности по этим кредитам.

Жилье же напротив имеет всего 6.9% клиентов с задолженностями.

Ну и самый интересный вывод это уровень дохода. Как оказалось уровень дохода не влияет на факт выплаты кредита в срок.